In [ ]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *
data_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/"
output_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/per_comune/"

In [ ]:
for f in os.listdir(data_folder+'clean_data/ministero_finanze'):
    extract_zipfile(data_folder+f'raw/ministero_finanze/{f}', data_folder+f'raw/socioeconomic/redditi/{f[-8:-4]}')

---

In [ ]:
redditi = dict()
for a in list(range(2000, 2019)) + [2022, 2023]:
    if (a > 2007) and (a < 2012):
        col = 'Codice Istat'
    else:
        col = 'Codice Istat Comune'
    redditi[a] = pd.read_csv(
        data_folder+f'raw/socioeconomic/redditi/{a}/Redditi_e_principali_variabili_IRPEF_su_base_comunale_CSV_{a}.csv', 
        encoding='latin-1', 
        sep=';'
        ).apply(lambda x: set_city_code(x, col), axis=1)

In [ ]:
for aa in [2019, 2020, 2021]: 
        a = pd.read_csv(
                        data_folder+f'raw/socioeconomic/redditi/{aa}/Redditi_e_principali_variabili_IRPEF_su_base_comunale_CSV_{aa}.csv', 
                encoding='latin-1', 
                sep=';',
                index_col=None).reset_index()
        df = pd.DataFrame()
        for n, c in enumerate(a.columns[1:-1]):
                df[c] = a.iloc[:, n]
        redditi[aa] = df.apply(lambda x: set_city_code(x, 'Codice Istat Comune'), axis=1)

In [ ]:
for a, df in redditi.items():
    save_clean_data(df, f'{a}', output_folder=data_folder+'clean_data/redditi/')

---

In [ ]:
redditi=dict()
for a in list(range(2000, 2024)):
    redditi[a] = pd.read_csv(f'/nfs/home/genovese/thesis-wildfire-genovese/data/clean_data/redditi/{a}.csv')

In [ ]:
from functools import reduce

data = [list(df.columns) for df in redditi.values()]

# Step 1: Find the intersection of all the lists
intersection = reduce(lambda a, b: set(a) & set(b), data)

# Step 2: Remove the intersection from each list
result = [[item for item in sublist if item not in intersection] for sublist in data]

for r in result:
    print(r)


In [ ]:
import re

for a, df in redditi.items():
    if df.columns[-1] == ' ':
        df.drop(columns=[' '], inplace=True)
    if "Codice Istat Regione" in df.columns:
        df.drop(columns=["Codice Istat Regione"], inplace=True)
    if "Codice Istat" in df.columns:
        df.rename(columns={'Codice Istat': 'Codice Istat Comune'}, inplace=True)

    df.columns = df.columns.str.replace("'", '', regex=True)
    df.columns = df.columns.str.replace("minore di zero euro", 'minore o uguale a zero euro', regex=True)
    cols = list(df.columns)
    renaming = dict.fromkeys(cols)
    for c in cols:
        if '-' in c:            
            split = c.split(' - ')
            split[0] = split[0].rstrip()
            split[1] = split[1].rstrip()
            pre = split[0]
            if'(' in pre:
                split[0] = re.sub(r"\([^)]*\)", "", pre).strip().rstrip()
            word = split[1]
            if 'Ammontare' in word:
                split[1] ='Ammontare in euro'
            
            new_col = split[0] + ' - ' + split[1]
            if 'contabilità' in new_col:
                new_col = new_col.replace('contabilità', 'contabilita')
            if 'IRPEF' in new_col:
                new_col = new_col.replace('IRPEF', '')
                split2 = new_col.split(' - ')
                split2[0] = split2[0].rstrip()
                split2[1] = split2[1].rstrip()
                new_col = split2[0] + ' - ' + split2[1]
            if ('dipendente' in new_col) and ('assimilati' not in new_col):
                split3 = new_col.split(' - ')
                split3[0] = split3[0].rstrip() + ' e assimilati'
                split3[0] = split3[0].rstrip()
                split3[1] = split3[1].rstrip()
                new_col = split3[0] + ' - ' + split3[1]
        else:
            new_col = c
        cols[cols.index(c)] = new_col

    df.columns = cols
        



In [ ]:
redditi[2012].rename(columns={'Reddito spettanza imprenditore ordinaria - Frequenza': 'Reddito di spettanza dellimprenditore in contabilita ordinaria - Frequenza',
                              'Reddito spettanza imprenditore semplificata - Frequenza': 'Reddito di spettanza dellimprenditore in contabilita semplificata - Frequenza',
                              'Reddito spettanza imprenditore ordinaria - Ammontare in euro': 'Reddito di spettanza dellimprenditore in contabilita semplificata - Ammontare in euro',
                              'Reddito spettanza imprenditore semplificata - Ammontare in euro': 'Reddito di spettanza dellimprenditore in contabilita ordinaria - Ammontare in euro'}, inplace=True)

In [ ]:
for a, df in redditi.items():
    df.columns = df.columns.str.replace('\xa0', ' ')

In [ ]:
data = [list(df.columns) for df in redditi.values()]

# Step 1: Find the intersection of all the lists
intersection = reduce(lambda a, b: set(a) & set(b), data)

# Step 2: Remove the intersection from each list
result = enumerate([[item for item in sublist if item not in intersection] for sublist in data])

for n, r in result:
    print(n+2000, r)

In [ ]:
for a, df in redditi.items():
    save_clean_data(df.drop(columns=df.columns[0]), f'{a}', output_folder=data_folder+'clean_data/redditi/')

---

In [ ]:
proporzioni_pensione = pd.DataFrame(
        {
        'Codice Istat Comune': redditi[2002]['Codice Istat Comune'], 
        'Reddito da pensione - Ammontare in euro': redditi[2002]['Reddito da pensione - Ammontare in euro']/redditi[2002]['Reddito da lavoro dipendente e assimilati - Ammontare in euro'],
        'Reddito da pensione - Frequenza': redditi[2002]['Reddito da pensione - Frequenza']/redditi[2002]['Reddito da lavoro dipendente e assimilati - Frequenza']
    })
proporzioni_pensione